# Biblioteki

In [2]:
import requests
import pandas as pd

Wczytanie domyślnych danych o książkach

In [3]:
test_title = "Classical Mythology"

In [23]:
books = pd.read_csv("./books_data/books.csv",low_memory=False)
#books.head()
#books.columns

##### sprawdziłem, *genres* może również odnosić się do książek

In [24]:
def get_book_description(title):

    google_books_api = f"https://www.googleapis.com/books/v1/volumes?q={title}"

    data = requests.get(google_books_api).json()
    
    if "items" in data:
        book_info = data["items"][0]["volumeInfo"]
        
        if "description" in book_info:
            return book_info["description"].lower()
    return "NONE"

def get_book_genre(title):

    google_books_api = f"https://www.googleapis.com/books/v1/volumes?q={title}"

    data = requests.get(google_books_api).json()
    
    if "items" in data:
        book_info = data["items"][0]["volumeInfo"]
        
        if "description" in book_info:
            return book_info["categories"]
    return "NONE"

#print(get_book_description(test_title))
#print(get_book_genre(test_title))

# TUTAJ ZMIENIĆ w  BOOKS_TEST NA BOOKS

In [20]:
books_description_genres = pd.DataFrame(columns=['Book-Title', 'Book_Description', 'Book-Genre'])

for index, row in books.iterrows():
    title = row['Book-Title']

    try:
        description = get_book_description(title)
        genre = get_book_genre(title)
        print(index)
        
        #print(title + " , i tu jakaś deskrypcja\n")
        #print(title + " " + genre[0])
    
    except Exception as e:
        # W przypadku błędu, przypisz None do description
        print(f"Błąd podczas pobierania opisu dla książki {title}: {str(e)}")
        description = None
        genre = None

    add_book = pd.DataFrame({'Book-Title': title, 'Book_Description': description, 'Book-Genre': genre})
    
    books_description_genres = pd.concat([books_description_genres,add_book], ignore_index=False)

#books_description_genres

0
1
2
3
4


,Book-Title,Book_Description,Book-Genre
0,Classical Mythology,"classical mythology, sixth edition, continues ...","Mythology, Classical"
0,Clara Callan,"in a small town in canada, clara callan reluct...",Fiction
0,Decision in Normandy,the strategy and planning behind d-day: “the b...,History
0,Flu: The Story of the Great Influenza Pandemic...,"the fascinating, true story of the world's dea...",Social Science
0,The Mummies of Urumchi,"an absorbing exploration of the mysterious, pe...",History


In [22]:
books_description_genres.to_csv("./books_data/books_info.csv", index=False)